In [1]:
import os 
import numpy as np
import json
from math import sqrt
from scipy.spatial import ConvexHull
from matplotlib import pyplot as plt
import cv2
import yaml

In [52]:
jsonexample = json.load(open("c/WIN_20230130_08_31_47_Pro.json"))
shape = jsonexample["shapes"]
flatlist = []
labelslist = []
with open("labels.yaml") as file:
    labellist = yaml.load(file, Loader=yaml.FullLoader)

labelcount = 0
bundle = []
labels = []
for i in range(len(shape)):
    label = labellist.index(shape[i]["label"])
    labels.append([str(label)])
    for points in shape[i]["points"]:
        xNorm = points[0]/jsonexample["imageWidth"]
        yNorm = points[1]/jsonexample["imageHeight"]
        labels[i].append(str(xNorm))
        labels[i].append(str(yNorm))

print(labels[0])

with open("testLabel.txt", "w") as file:
    for i in range(len(labels)):
        for x in labels[i]:
            file.write(x)
            file.write(" ")
        file.write('\n')
        


    


['12', '0.22232680722891568', '0.8168228469433289', '0.21730672690763056', '0.8045515394912985', '0.21354166666666669', '0.7967425256581883', '0.21354166666666669', '0.7833556448014279', '0.2110316265060241', '0.7398482820169567', '0.20852158634538157', '0.5234270414993307', '0.2110316265060241', '0.38844265952699686', '0.21416917670682734', '0.24118697010263274', '0.21918925702811248', '0.17759928603302103', '0.22358182730923695', '0.16086568496207054', '0.2286019076305221', '0.1552878179384204', '0.34155371485943775', '0.1441320838911201', '0.3553589357429719', '0.1441320838911201', '0.3522213855421687', '0.6494868362338242', '0.35912399598393574', '0.8023203926818384', '0.3572414658634538', '0.825747434181169', '0.34594628514056225', '0.8290941543953592']


In [ ]:

def min_bounding_rect(points):
    from scipy.ndimage.interpolation import rotate
    pi2 = np.pi/2.
    hull_points = points[ConvexHull(points).vertices]
    
    edges = np.zeros((len(hull_points)-1,2))
    edges = hull_points[1:] - hull_points[:-1]
    
    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])
    
    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)
    
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles - pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T
    rotations = rotations.reshape((-1,2,2))
    
    rot_points = np.dot(rotations, hull_points.T)
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)
    
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)
    
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]
    coords = (x1,y1,x2,y2)
    return coords
    
rect = min_bounding_rect(flatlist)

In [ ]:
def xyxy2xywh(x):
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = (x[:, 0] + x[:, 2]) / 2
    y[:, 1] = (x[:, 1] + x[:, 3]) / 2

In [3]:
x1 = abs(int(rect[0]))
y1 = abs(int(rect[1]))
x2 = abs(int(rect[2]))
y2 = abs(int(rect[3]))
testimg = cv2.imread("c/WIN_20230130_08_31_47_Pro.jpg", 1)
rectimg = cv2.rectangle(testimg,(x1,y1), (x2,y2), (255,0,0),3)
cv2.imwrite("c/testrect.png", rectimg)


True